In [1]:
import numpy as np
import Utils as ut
import KernelFile as kf
import CortesAlignmentFile as ca
import MKL_SROLAnonNeg as mkl
import mySampler as ms
import myGridSearch as mgs

In [2]:
from sklearn import datasets
# make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, 
#                     n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
#                     hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
ds, label = datasets.make_classification(n_samples=300, n_features=30, n_informative=10, n_redundant=0, n_classes=2)

In [3]:
np.where(label==0)
label[np.where(label==0)]=-1

In [4]:
def random_sampling(X, y, n_samples):
    from sklearn.model_selection import StratifiedShuffleSplit
    return next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))

In [5]:
tr_idx, ts_idx = random_sampling(ds, label, n_samples=0.25)

ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = ds[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

l_tr = label[tr_idx]
l_ts = label[ts_idx]

ds_list = [ds1, ds2, ds3]
tr_list = [ds1_tr, ds2_tr, ds3_tr]
ts_list = [ds1_ts, ds2_ts, ds3_ts]

In [6]:
# wrapper testing

kernel_type = [{'linear':[0], 'polynomial':[2, 8, 12], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0], 'polynomial':[20, 30, 50], 'gaussian':[1, 3, 5]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25)

In [7]:
sampler.sample(kernel_type, estimator, ds_list, label, valid_fold = 3, verbose=True)

1 split out of 4 ...
	Working on config 1 of 3: {'linear': [0], 'polynomial': [2, 8, 12], 'gaussian': [0.1, 0.5, 0.7]}
	Working on config 2 of 3: {'linear': [0], 'polynomial': [20, 30, 50], 'gaussian': [1, 3, 5]}
	Result of 0:
	[linear:0, polynomial:2, gaussian:0.1, ]
[linear:0, polynomial:8, gaussian:0.5, ]
[linear:0, polynomial:2, gaussian:0.1, ]

	[linear:0, polynomial:20, gaussian:1, ]
[linear:0, polynomial:20, gaussian:3, ]
[linear:0, polynomial:50, gaussian:5, ]

2 split out of 4 ...
	Working on config 1 of 3: {'linear': [0], 'polynomial': [2, 8, 12], 'gaussian': [0.1, 0.5, 0.7]}
	Working on config 2 of 3: {'linear': [0], 'polynomial': [20, 30, 50], 'gaussian': [1, 3, 5]}
	Result of 1:
	[linear:0, polynomial:12, gaussian:0.1, ]
[linear:0, polynomial:8, gaussian:0.7, ]
[linear:0, polynomial:12, gaussian:0.1, ]

	[linear:0, polynomial:50, gaussian:3, ]
[linear:0, polynomial:20, gaussian:3, ]
[linear:0, polynomial:50, gaussian:5, ]

3 split out of 4 ...
	Working on config 1 of 3: {'

In [ ]:
# GRIDSEARCH CV

kernel_type = {'linear':[0], 'polynomial':[2, 8, 12], 'gaussian':[0.1, 0.5, 0.7]}
estimator = ca.centeredKernelAlignment
gs = mgs.myGridSearchCV(estimator, kernel_type, fold = 3).fit(tr_list, l_tr)

In [ ]:
sel_accuracy, sel_kWrapp, weights = gs.transform(tr_list, verbose=True)

In [ ]:
weights

In [ ]:
pred = sel_kWrapp.predict(ts_list, weights, l_tr)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(l_ts, pred)

In [ ]:
# SimpleMKL APPROACH

# kernel_type = {'linear': [0], 'polynomial': [2,3,4], 'gaussian':[0.3, 0.5, 0.7]}
kernel_type = {'linear': 0, 'polynomial': 4, 'gaussian': 0.5}

smkl = mkl.MKL_simpleSrola(kernel_type, 10, 0.01)
smkl.fit(tr_list, l_tr)

In [ ]:
weights = smkl.learnEta(tr_list)

In [ ]:
x1 = np.arange(12).reshape((4, 3))
x2 = np.arange(3)
np.multiply(x1, x2)

In [ ]:
a = np.array([[1,2],[3,4]])
print(a.ravel().reshape(-1,1).shape)